In [1]:
import sys

sys.path.append("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning")
import os
os.chdir("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning")

import NLP_utils.preprocessing as nlp_processing
from RL_for_NLP.text_environments import TextEnvClfWithBertTokens, TextEnvClf, TextEnvClfForBertModels
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm

from RL_for_NLP.text_data_pools import PartialReadingDataPoolWithTokens, PartialReadingDataPoolWithBertTokens
import reinforce_algorithm_utils as rl_monte_carlo
import policy_networks as pn

import torch as th
from torch.optim import Adam
import mlflow
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm

import actor_critic_algortihm_utils as a2c_utils 

from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3 import DQN

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import json
from collections import Counter
from tqdm import tqdm

Using TensorFlow backend.


In [6]:
data = nlp_processing.openDfFromPickle("NLP_datasets/ag_news/ag_news_train_distilbert-base-uncased.pkl")
pool = PartialReadingDataPoolWithBertTokens(data, "text", "label", 512, 50, mask = True)
env = TextEnvClfForBertModels(pool, 30522, int(1e+5), "score", True)

Maximum sentence length in pool: 512


Padding the data and populating samples...: 100%|██████████| 96000/96000 [00:08<00:00, 10835.31it/s]

---- Inside constructor ---------
Input id shape/type:  (50,) <class 'numpy.ndarray'>
Attn mask shape/type:  (50,) <class 'numpy.ndarray'>
------------- End Constructor-------------


In [3]:
env.current_state

{'input_id': array([  101, 10556, 24175,  2072, 13980,  2602,  8320,  2000, 21250,
         7041,  9455,  2343, 24811, 10556, 24175,  2072,  2038,  2187,
         2010,  4600, 13313,  7328,  1999, 21073,  2005,  2010,  2034,
         2602,  8320,  1999,  1996,  2197,  2733,  1997, 18524,  2005,
         2023,  5095,  2034,  2412,  3622,  3864,   102,     0,     0,
            0,     0,     0,     0,     0]),
 'attn_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0])}

In [7]:
from transformers import DistilBertModel, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [4]:
inputs2 = env.current_state
inputs2

{'input_id': array([  101,  3027,  3366,  4978,  2733,  2659,  2327,  6661,  2031,
         5357,  2000,  2093,  2733,  2659,  2044,  2019,  2220,  5376,
         2006,  1996,  2067,  1997,  3893,  5971,  2739,  3478,  2000,
         8587, 11071,  1998,  6428,  2084,  3517,  7027,  4341,  2951,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]),
 'attn_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])}

In [19]:
th.tensor(inputs2["input_id"]).shape

torch.Size([50])

In [13]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
inputs["input_ids"].shape

torch.Size([1, 8])

In [8]:
out = distilbert(th.tensor(env.current_state["input_id"]).unsqueeze(0), th.tensor(env.current_state["attn_mask"]).unsqueeze(0))

In [11]:
out

BaseModelOutput(last_hidden_state=tensor([[[-0.0948, -0.0587,  0.1873,  ..., -0.2871,  0.5707,  0.1970],
         [ 0.2357, -0.1329,  0.6317,  ...,  0.3515,  0.5148, -0.2955],
         [-0.2966, -0.3269,  0.6578,  ..., -0.0205,  0.4010, -0.0548],
         ...,
         [ 0.1536,  0.0169,  0.3824,  ..., -0.0989,  0.1536, -0.0054],
         [ 0.0532,  0.1062,  0.2115,  ..., -0.0597,  0.1105, -0.0880],
         [ 0.2399, -0.1958,  0.3171,  ..., -0.0178,  0.1243,  0.0880]]],
       grad_fn=<NativeLayerNormBackward>), hidden_states=None, attentions=None)

In [26]:
distilbert.add_module(th.nn.Flatten())
distilbert.add_module(th.nn.Linear(50*768, 64))


TypeError: add_module() missing 1 required positional argument: 'module'

In [4]:
policy = a2c_utils.DistibertActorCriticPolicy(50, 4)

In [5]:
policy(th.tensor(env.current_state["input_id"]).unsqueeze(0), th.tensor(env.current_state["attn_mask"]).unsqueeze(0))

(tensor([[0.2148, 0.2861, 0.2463, 0.2528]], grad_fn=<SoftmaxBackward>),
 tensor([[0.1418]], grad_fn=<AddmmBackward>))